## Checking the error Data for 2021 Jan and May

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import datetime

/Users/endy/.pyenv/versions/3.10.6/envs/crypto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torch

In [3]:
!pip install protobuf==3.20.0

## Step 1 - Importing Bitcoin Tweet Data to begin analysing the model

In [4]:
# Kaggle - 2021Q1
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q1_reduced.csv')

# Kaggle - 2021Q2
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

# Kaggle - 2021Q3
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

# Kaggle - 2021Q4
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

In [5]:
# Kaggle - 202101, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202101_reduced.csv')
# Kaggle - 202102, issue
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202102_reduced.csv')
# data = data.drop([8120]) #error data in row = 8120
# Kaggle - 202103, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202103_reduced.csv')
# Kaggle - 202104, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202104_reduced.csv')
# Kaggle - 202105, issue
data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202105_reduced.csv')
data = data.drop([6690]) #error data in row = 6690
# Kaggle - 202106, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202106_reduced.csv')
# Kaggle - 202107, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202107_reduced.csv')
# Kaggle - 202108, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202108_reduced.csv')
# Kaggle - 202109, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202109_reduced.csv')
# Kaggle - 202110, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202110_reduced.csv')
# Kaggle - 202111, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202111_reduced.csv')
# Kaggle - 202112, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202112_reduced.csv')

In [6]:
data

,Unnamed: 0,datetime,username,text,has_#bitcoin,date
0,61685,2021-05-01 22:09:58+00:00,Nihlahm14,Good project\n#punkprotocol #airdrop #giveaway...,True,2021-05-01
1,61686,2021-05-01 22:09:55+00:00,wearecryptoarmy,Demand for Cryptocoin Mixing Rises Significant...,True,2021-05-01
2,61687,2021-05-01 22:09:54+00:00,Devlishh,@moonxbtee you are seriously asking me which i...,True,2021-05-01
3,61688,2021-05-01 22:09:54+00:00,dogecrypty,"We rise again, stronger than ever...\n\n#dodge...",True,2021-05-01
4,61689,2021-05-01 22:09:46+00:00,Alaskacryptogi1,#Bitcoin this is absolutely irony. Every gover...,True,2021-05-01
...,...,...,...,...,...,...
20014,81699,2021-05-31 06:00:02+00:00,BitcoinBellyB,"The price of\nSpaghetti alla Carbonara\nis 31,...",True,2021-05-31
20015,81700,2021-05-31 06:00:01+00:00,takeoff_tech,"【Our Logo】\n\nEveryone’s dreams, all kinds of ...",True,2021-05-31
20016,81701,2021-05-31 06:00:01+00:00,PlatinumCrypto,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,True,2021-05-31
20017,81702,2021-05-31 06:00:01+00:00,lemalive,Monday morning.\nJust woke up to win a fucking...,True,2021-05-31


In [7]:
data = data[['text', 'date']]

In [8]:
data

,text,date
0,Good project\n#punkprotocol #airdrop #giveaway...,2021-05-01
1,Demand for Cryptocoin Mixing Rises Significant...,2021-05-01
2,@moonxbtee you are seriously asking me which i...,2021-05-01
3,"We rise again, stronger than ever...\n\n#dodge...",2021-05-01
4,#Bitcoin this is absolutely irony. Every gover...,2021-05-01
...,...,...
20014,"The price of\nSpaghetti alla Carbonara\nis 31,...",2021-05-31
20015,"【Our Logo】\n\nEveryone’s dreams, all kinds of ...",2021-05-31
20016,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,2021-05-31
20017,Monday morning.\nJust woke up to win a fucking...,2021-05-31


## Step 2 - Cleaning the data

In [9]:
# I have changed this to remove more information

def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [10]:
#Create a dummy data
data['process_text'] = data.text
data['negative_bert'] = data.text
data['neutral_bert'] = data.text
data['positive_bert'] = data.text

#use the preprocess to clean the data
data['process_text'] = data['text'].apply(preprocess)

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10499/1973951134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['process_text'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10499/1973951134.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['negative_bert'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10499/1973951134.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [11]:
data

,text,date,process_text,negative_bert,neutral_bert,positive_bert
0,Good project\n#punkprotocol #airdrop #giveaway...,2021-05-01,Good project #punkprotocol #airdrop #giveaway ...,Good project\n#punkprotocol #airdrop #giveaway...,Good project\n#punkprotocol #airdrop #giveaway...,Good project\n#punkprotocol #airdrop #giveaway...
1,Demand for Cryptocoin Mixing Rises Significant...,2021-05-01,Demand for Cryptocoin Mixing Rises Significant...,Demand for Cryptocoin Mixing Rises Significant...,Demand for Cryptocoin Mixing Rises Significant...,Demand for Cryptocoin Mixing Rises Significant...
2,@moonxbtee you are seriously asking me which i...,2021-05-01,you are seriously asking me which is my favou...,@moonxbtee you are seriously asking me which i...,@moonxbtee you are seriously asking me which i...,@moonxbtee you are seriously asking me which i...
3,"We rise again, stronger than ever...\n\n#dodge...",2021-05-01,"We rise again, stronger than ever... #dodgeco...","We rise again, stronger than ever...\n\n#dodge...","We rise again, stronger than ever...\n\n#dodge...","We rise again, stronger than ever...\n\n#dodge..."
4,#Bitcoin this is absolutely irony. Every gover...,2021-05-01,#Bitcoin this is absolutely irony. Every gover...,#Bitcoin this is absolutely irony. Every gover...,#Bitcoin this is absolutely irony. Every gover...,#Bitcoin this is absolutely irony. Every gover...
...,...,...,...,...,...,...
20014,"The price of\nSpaghetti alla Carbonara\nis 31,...",2021-05-31,"The price of Spaghetti alla Carbonara is 31,87...","The price of\nSpaghetti alla Carbonara\nis 31,...","The price of\nSpaghetti alla Carbonara\nis 31,...","The price of\nSpaghetti alla Carbonara\nis 31,..."
20015,"【Our Logo】\n\nEveryone’s dreams, all kinds of ...",2021-05-31,"【Our Logo】 Everyone’s dreams, all kinds of dr...","【Our Logo】\n\nEveryone’s dreams, all kinds of ...","【Our Logo】\n\nEveryone’s dreams, all kinds of ...","【Our Logo】\n\nEveryone’s dreams, all kinds of ..."
20016,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,2021-05-31,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...
20017,Monday morning.\nJust woke up to win a fucking...,2021-05-31,Monday morning. Just woke up to win a fucking ...,Monday morning.\nJust woke up to win a fucking...,Monday morning.\nJust woke up to win a fucking...,Monday morning.\nJust woke up to win a fucking...


## Step 3 - analyse the language and sentiments by pretrained model

In [12]:
!pyenv local crypto

In [13]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"ElKulako/cryptobert"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
config_bert = AutoConfig.from_pretrained(MODEL_bert)


# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
model_bert.config.max_position_embeddings = 512

In [14]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

In [21]:
starting_num = 6650
ending_num = 6652

for i in range(25):
    try:
        data['process_text'][starting_num : ending_num].apply(scores_bert)
        print(f'{starting_num} to {ending_num} is going to work!')
        working = True
    except:
        print(f'{starting_num} to {ending_num} has not processed correctly')
    if working == True:
        result_csv = pd.DataFrame(data['process_text'][starting_num : ending_num])
        result_csv['text'] = result_csv['process_text'].apply(scores_bert)
        print(f'{starting_num} to {ending_num} has worked!')
        file_name = f"{starting_num}_{ending_num}_twitter_comments.csv"
        result_csv.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")
        print(f'{starting_num} to {ending_num} saved!')
    starting_num += 2
    ending_num += 2

6650 to 6652 is going to work!
6650 to 6652 has worked!
6650 to 6652 saved!
6652 to 6654 is going to work!
6652 to 6654 has worked!
6652 to 6654 saved!
6654 to 6656 is going to work!
6654 to 6656 has worked!
6654 to 6656 saved!
6656 to 6658 is going to work!
6656 to 6658 has worked!
6656 to 6658 saved!
6658 to 6660 is going to work!
6658 to 6660 has worked!
6658 to 6660 saved!
6660 to 6662 is going to work!
6660 to 6662 has worked!
6660 to 6662 saved!
6662 to 6664 is going to work!
6662 to 6664 has worked!
6662 to 6664 saved!
6664 to 6666 is going to work!
6664 to 6666 has worked!
6664 to 6666 saved!
6666 to 6668 is going to work!
6666 to 6668 has worked!
6666 to 6668 saved!
6668 to 6670 is going to work!
6668 to 6670 has worked!
6668 to 6670 saved!
6670 to 6672 is going to work!
6670 to 6672 has worked!
6670 to 6672 saved!
6672 to 6674 is going to work!
6672 to 6674 has worked!
6672 to 6674 saved!
6674 to 6676 is going to work!
6674 to 6676 has worked!
6674 to 6676 saved!
6676 to 6678

RuntimeError: The expanded size of the tensor (672) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 672].  Tensor sizes: [1, 514]

In [25]:
data[6690:6691]

,text,date,process_text,negative_bert,neutral_bert,positive_bert
6690,We can do this army shiba 🏳️‍🌈🏳️‍⚧️🇺🇳🇦🇫🇿🇦🇦🇴🇦🇩🇩...,2021-05-14,We can do this army shiba 🏳️‍🌈🏳️‍⚧️🇺🇳🇦🇫🇿🇦🇦🇴🇦🇩🇩...,We can do this army shiba 🏳️‍🌈🏳️‍⚧️🇺🇳🇦🇫🇿🇦🇦🇴🇦🇩🇩...,We can do this army shiba 🏳️‍🌈🏳️‍⚧️🇺🇳🇦🇫🇿🇦🇦🇴🇦🇩🇩...,We can do this army shiba 🏳️‍🌈🏳️‍⚧️🇺🇳🇦🇫🇿🇦🇦🇴🇦🇩🇩...


In [24]:
data['text'][6690:6691] = data['process_text'][6690:6691].apply(scores_bert)

RuntimeError: The expanded size of the tensor (672) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 672].  Tensor sizes: [1, 514]

In [204]:
for i in range(len(data)):    
    data['negative_bert'][i] = data['text'][i][0]
    data['neutral_bert'][i] = data['text'][i][1]
    data['positive_bert'][i] = data['text'][i][2]

In [205]:
data = data[['date', 'process_text', 'negative_bert', 'neutral_bert','positive_bert']]

In [64]:
problem = str(data[8120:8121]['text']).split()

In [66]:
problem[2]

'\\n\\n👉🏿👇🏿👇🏿👇🏿👇🏿👇🏿👇🏿👇🏿👇🏿👇🏿👈🏿\\n👉🏿👇🏾👇🏾👇🏾👇...'

## Step 4: Count the total number of labels/scores (positive, negative vs neutral) by date

In [207]:
grouped_data = pd.DataFrame(data.groupby(['date'])[['negative_bert', 'neutral_bert', 'positive_bert']].sum().fillna(0).reset_index())
grouped_data = grouped_data.rename_axis("", axis="columns")

In [208]:
grouped_data

,date,negative_bert,neutral_bert,positive_bert
0,2021-12-01,33.079273,334.769775,253.150803
1,2021-12-02,29.942642,304.941711,242.115707
2,2021-12-03,48.248924,314.411469,239.339539
3,2021-12-04,111.281281,466.295471,434.422882
4,2021-12-05,35.309681,278.630463,234.059875
5,2021-12-06,56.319256,336.050903,305.629791
6,2021-12-07,39.934368,238.508957,175.556717
7,2021-12-08,34.813095,242.072983,179.113892
8,2021-12-09,46.616699,245.123703,173.259750
9,2021-12-10,50.885303,257.110504,205.004272


## Step 5 - Download the data

In [209]:
start_date = grouped_data['date'][0]
# start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
start_date

'2021-12-01'

In [210]:
end_date = grouped_data['date'][len(grouped_data)-1]
# end_date_str = datetime.datetime.strftime(end_date, "%Y-%m-%d")
end_date

'2021-12-31'

In [211]:
file_name = f"{start_date}_{end_date}_twitter_comments.csv"
data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")

In [212]:
file_name = f"{start_date}_{end_date}_twitter_transferlearning.csv"
grouped_data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")

In [84]:
test = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202102_reduced.csv')
test

,Unnamed: 0,datetime,username,text,has_#bitcoin,date
0,14003,2021-02-01 22:09:58+00:00,mferreira680,Wohhhh is great!!!!! \nPls give some suvenir ...,True,2021-02-01
1,14004,2021-02-01 22:09:33+00:00,mbs_FiTTrader,Well @elonmusk made a joke about #dodgecoin an...,True,2021-02-01
2,14005,2021-02-01 22:09:32+00:00,oneredwanonly,It's about inequality and unfair markets. Taxe...,True,2021-02-01
3,14006,2021-02-01 22:09:26+00:00,bmurphypointman,#affiliatemarketing #affiliate #travel #touris...,True,2021-02-01
4,14007,2021-02-01 22:09:25+00:00,bmurphypointman,#tumblr #twitter #gifting #facebook #instagram...,True,2021-02-01
...,...,...,...,...,...,...
16214,30217,2021-02-28 06:00:00+00:00,tulipan81,Bitcoin mined blocks distribution per pool in ...,True,2021-02-28
16215,30218,2021-02-28 06:00:00+00:00,tulipan81,Bitcoin free space in mined blocks in last wee...,True,2021-02-28
16216,30219,2021-02-28 06:00:00+00:00,tulipan81,Bitcoin free space in mined blocks in last day...,True,2021-02-28
16217,30220,2021-02-28 06:00:00+00:00,tulipan81,Bitcoin mined blocks distribution per pool in ...,True,2021-02-28


In [30]:
test = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202105_reduced.csv')

In [31]:
test[6690:6691]

,Unnamed: 0,datetime,username,text,has_#bitcoin,date
6690,68375,2021-05-14 14:07:07+00:00,Titeeeb,We can do this army shiba 🏳️‍🌈🏳️‍⚧️🇺🇳🇦🇫🇿🇦🇦🇴🇦🇩🇩...,True,2021-05-14


In [33]:
test.drop([6690])[6685:6695]

,Unnamed: 0,datetime,username,text,has_#bitcoin,date
6685,68370,2021-05-14 14:07:50+00:00,SoodiniR,Millions of people across the world living in ...,True,2021-05-14
6686,68371,2021-05-14 14:07:46+00:00,dotheextra,Locked in my profit from ADA ✅ #btc #bitcoin h...,True,2021-05-14
6687,68372,2021-05-14 14:07:36+00:00,RTANGEL11,@Yamiche The Gops attempting to use stefanik t...,True,2021-05-14
6688,68373,2021-05-14 14:07:31+00:00,Dark_Ages_2020,#cryptocurrency mining is swallowing up the su...,True,2021-05-14
6689,68374,2021-05-14 14:07:29+00:00,5Ksana,@NickyPapersNFT My cat Leo loves #bitcoin more...,True,2021-05-14
6691,68376,2021-05-14 14:07:02+00:00,realLooms,@defichain is a #bitcoin fork that doesn't us...,True,2021-05-14
6692,68377,2021-05-14 14:06:56+00:00,terr547,@girlgone_crypto And #bitcoin still barely abo...,True,2021-05-14
6693,68378,2021-05-14 14:06:54+00:00,achim1235,@AltcoinDailyio Check out the one and only „De...,True,2021-05-14
6694,68379,2021-05-14 14:06:51+00:00,Bouillaka_CF,Do you think @elonmusk bought the dip in secre...,True,2021-05-14
6695,68380,2021-05-14 14:06:42+00:00,Dr_CoreyWilson,@DocumentingBTC If #Bitcoin is really so bad f...,True,2021-05-14
